## Problem

- zip image --> geotif image with order bands: B1, B2, B3,..., B12 and has name: ..,GREEN resolution 10m, 
BLUE resolution 10m, RED.....

## Input

In [6]:
path = "/Users/mrdung/datacube/datacube_data/data/download/S2B_MSIL2A_20200301T031639_N0214_R118_T48PXS_20200301T072900.zip"

## Gather infor about Input

#1 Infor about Image

In [ ]:
!gdalinfo /Users/mrdung/datacube/datacube_data/data/download/S2B_MSIL2A_20200301T031639_N0214_R118_T48PXS_20200301T072900.zip

#2 Info Subdataset

In [ ]:
import gdal, gdalconst
import subprocess


dataset = gdal.Open(path, gdal.GA_ReadOnly)
list_sub_dataset = dataset.GetSubDatasets()
list_sub_dataset

## Solution

### Sol 1

- Step 1: Get description of the original image
- Step 2: Set descriptions to the destinational image

In [97]:
import numpy as np
import rasterio


def get_tag_and_descriptions(path):
    src = rasterio.open(path)
    tags = [src.tags(i+1) for i in range(src.count)]
    return src.descriptions, tags

def get_subdataset_tags_descriptions(path):
    """ return tuple of lists: (list_descriptions, list_tags, metadata(dict)) for each band"""
    subdatasets = src.subdatasets
    band_tag_des = [get_tag_and_descriptions(sub__) for sub__ in subdatasets]
    
    # change from list((tuple, list)) of each subdataset to target format
    band_tag_des = np.array(band_tag_des)
    return list(np.concatenate(band_tag_des[:, 0])), list(np.concatenate(band_tag_des[:, 1])), src.tags()

def set_descriptions_tags(dest_path, source_path):
    r = get_subdataset_tags_descriptions(source_path)
    with rasterio.open(dest_path, mode='r+') as dest_src:
        descriptions, tags, metadata = r
        for i in range(dest_src.count):
            dest_src.set_band_description(i+1, descriptions[i])
            dest_src.update_tags(i+1, **tags[i])
        dest_src.update_tags(**metadata)
        
set_descriptions_tags(
    "/Users/mrdung/datacube/datacube_data/data/download/dung.tif",
    path
)